Initialization of ES

In [1]:
from elasticsearch import Elasticsearch

client = Elasticsearch(['host.docker.internal'])
indice = "syslog-2021-02-24,syslog-2021-02-25,syslog-2021-02-26"

1. Query All 

In [ ]:
query = {
    "query": {
        "match_all": {}
    }
}

res = client.search(index=indice, body=query, scroll='100m', size=10000)

print("Got %d Hits:" % res['hits']['total']['value'])
    
sid = res['_scroll_id']
scroll_size = len(res['hits']['hits'])

while scroll_size > 0:
    "Scrolling..."
    print(scroll_size)
    data = client.scroll(scroll_id=sid, scroll='2m')
    # Update the scroll ID
    sid = data['_scroll_id']
    # Get the number of results that returned in the last scroll
    scroll_size = len(data['hits']['hits'])

2. Match Query

In [ ]:
query = {
    "query": {
        "match": {
            "hostname":"for.org"
        }
    }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

3. Multi Match

In [ ]:
query = {
    "query": {
        "multi_match": {
            "query": "up.com ahmadajmi", 
            "fields":["hostname", "application"]
        }
    }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

4. String Query

In [ ]:
query = {
  "query": {
    "query_string": {
      "query": "(for.org) AND (pretty breath) "
    }
  }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

5. Term Query

In [ ]:
query = {
   "query":{
      "term":{"message":"pretty"}
   }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

6. Range Query

In [ ]:
query = {
   "query":{
      "range":{
         "version":{
            "gte":2
         }
      }
   }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

7. Exist Query

In [ ]:
query = {
  "query": {
    "exists": {
      "field": "application"
    }
  }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

8. Regex Query
https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-regexp-query.html

In [2]:
query = {
  "query": {
    "regexp": {
      "hostname": {
        "value": "up.*",
        "flags": "ALL",
        "max_determinized_states": 10000,
        "rewrite": "constant_score"
      }
    }
  }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

Got 10000 Hits:
2021-02-24T23:44:56.200Z <29>1 2021-02-24T23:44:56.200Z up.org shaneIxD 2747 ID774 - There's a breach in the warp core, captain
2021-02-24T23:44:56.250Z <43>1 2021-02-24T23:44:56.250Z up.org devankoshal 3940 ID420 - Pretty pretty pretty good
2021-02-24T23:45:55.395Z <49>2 2021-02-24T23:45:55.395Z up.de benefritz 8523 ID509 - You're not gonna believe what just happened
2021-02-24T23:45:55.435Z <90>2 2021-02-24T23:45:55.435Z up.us shaneIxD 7480 ID833 - We're gonna need a bigger boat
2021-02-24T23:45:55.695Z <122>1 2021-02-24T23:45:55.695Z up.com Karimmove 2258 ID67 - Take a breath, let it go, walk away
2021-02-24T23:45:55.705Z <162>1 2021-02-24T23:45:55.705Z up.com shaneIxD 9784 ID281 - Pretty pretty pretty good
2021-02-24T23:45:55.715Z <59>2 2021-02-24T23:45:55.715Z up.us devankoshal 5168 ID985 - Maybe we just shouldn't use computers
2021-02-24T23:45:55.825Z <135>1 2021-02-24T23:45:55.825Z up.us meln1ks 3570 ID610 - Pretty pretty pretty good
2021-02-24T23:45:55.906Z <104

9. Compount Query https://www.elastic.co/guide/en/elasticsearch/reference/current/compound-queries.html

In [ ]:
query = {
   "query": {
      "bool" : {
         "must" : {
            "term" : { "hostname" : "random.net" }
         },
         "should": {
            "term" : { "application" : "ahmadajmi" }
         },
         "minimum_should_match" : 1,
         "boost" : 1.0
      }
   }
}


res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

10. Count aggregation

In [ ]:
query = {
   "aggs":{
      "version_count":{
         "value_count":{
            "field":"version"
         }
      }
   }
}


res = client.search(index=indice, body=query )

print("Got %d Hits:" % res['hits']['total']['value'])

res['aggregations']

11. Cardinality aggregation

In [ ]:
query = {
  "aggs": {
    "my-agg-name": {
      "cardinality": {
        "field": "priority"
      }
    }
  }
}
    
res = client.search(index=indice, body=query, scroll='100m', size=10000)

print("Got %d Hits:" % res['hits']['total']['value'])

print(res['aggregations'])